In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from zipfile import ZipFile
file_name = "/content/drive/MyDrive/dataset_split.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')


Done


In [ ]:
from zipfile import ZipFile
file_name = "/content/drive/MyDrive/Shared.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
import numpy as np
import argparse
import dlib
import cv2
import os
import matplotlib.pyplot as plt
import imutils
from imutils import face_utils
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [ ]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('/content/Shared/shape_predictor_68_face_landmarks.dat')


In [ ]:
def get_data(direc):
    X =[]
    y = []
    data = []
    IMG_SIZE = 50
    categories = os.listdir(direc)
    for category in categories:
        path_link = os.path.join(direc, category)
        for image in os.listdir(path_link):
            image_array = cv2.imread(os.path.join(path_link, image),cv2.IMREAD_GRAYSCALE)
            image_array = cv2.resize(image_array, (IMG_SIZE, IMG_SIZE))
            X.append(image_array)
            y.append(int(category))
    return X,y



In [ ]:
X_train,y_train = get_data("/content/dataset_split/train")
X_test,y_test = get_data("/content/dataset_split/test")


In [ ]:
X_train = np.array(X_train).reshape(-1,50,50,1)
X_test = np.array(X_test).reshape(-1,50,50,1)
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

y_train = np.array(y_train).reshape(-1,1)
y_test = np.array(y_test).reshape(-1,1)
y_train = y_train.flatten()
y_test = y_test.flatten()

In [ ]:
train_dir = "/content/dataset_split/train"
test_dir = "/content/dataset_split/test"

In [ ]:
batch = 32
learning_rate = 0.0001
adam = Adam(lr=learning_rate)

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
callbacks = [
    EarlyStopping(monitor = 'val_loss',
                          min_delta = 0, #minimium amount of change to count as an improvement
                          patience = 10, # how many epochs to wait before stopping
                          verbose = 1,
                          restore_best_weights = True),
    ReduceLROnPlateau(factor=0.9, patience=10, min_lr=0.0001),
    ModelCheckpoint('EnsembleEyeModel.h5', verbose=1, save_best_only=True)#, save_weights_only=True
]

In [ ]:
#! pip uninstall tensorflow
#! pip install tensorflow==2.3.0

In [ ]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Average
eyeModel1 = tf.keras.models.load_model(r"/content/drive/MyDrive/NewEyeModel-0.969.h5")
eyeModel1._name = 'eyeModel1'
eyeModel2 = tf.keras.models.load_model(r"/content/drive/MyDrive/NewEyeModel-0.966.h5")
eyeModel2._name = 'eyeModel2'
eyeModel3 = tf.keras.models.load_model(r"/content/drive/MyDrive/NewEyeModel-0.968.h5")
eyeModel3._name = 'eyeModel3'

models = [eyeModel1, eyeModel2,eyeModel3]
model_input = Input(shape=(50, 50, 1))
model_outputs = [model(model_input) for model in models]
ensemble_output = Average()(model_outputs)
ensemble_model = Model(inputs=model_input, outputs=ensemble_output, name='ensemble')

ensemble_model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer=adam)
ensemble_model.save('EyeEnsemble.h5')

In [ ]:
eModel1 = tf.keras.models.load_model(r"/content/EyeEnsemble.h5")
eModel1._name = 'ensemble'

In [ ]:
(loss,acc) = eModel1.evaluate(X_test,y_test, verbose=0)
print('Model Evaluation')
print('Loss: %.3f' % loss)
print('Accuracy: %.3f' % acc)
print()

Model Evaluation
Loss: 0.078
Accuracy: 0.971



In [ ]:
(loss,acc) = ensemble_model.evaluate(X_test,y_test, verbose=0)
print('Model Evaluation')
print('Loss: %.3f' % loss)
print('Accuracy: %.3f' % acc)
print()

Model Evaluation
Loss: 0.078
Accuracy: 0.971



In [ ]:
(loss,acc) = eyeModel1.evaluate(X_test,y_test, verbose=0)
print('Model Evaluation')
print('Loss: %.3f' % loss)
print('Accuracy: %.3f' % acc)
print()

In [ ]:
(loss,acc) = eyeModel2.evaluate(X_test,y_test, verbose=0)
print('Model Evaluation')
print('Loss: %.3f' % loss)
print('Accuracy: %.3f' % acc)
print()